In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph_MNL"

print("--")


--


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [4]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph_MNL, 9LXqY2Fshgupvi9vQKDPRpdtxeMfTVMBXNCW5o8vtzKu, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [8]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , ["Airport", "CoolTown"]],
   ["ORD", "Chicago O-Hare"          , ["Airport", "FIBS"    ]],
#  ["MKE", "Milwaukee"               , "Airport"],
#  ["ORD", "Chicago O-Hare"          , "Airport"],
    
   ["SJC", "San Jose"                , ["Airport"]],
   ["DEN", "Denver"                  , ["Airport"]],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


+----+----------------+----------------+-------------------------+
|    | airport_code   | airport_name   | LABEL                   |
|----+----------------+----------------+-------------------------|
|  0 | MKE            | Milwaukee      | ['Airport', 'CoolTown'] |
|  1 | ORD            | Chicago O-Hare | ['Airport', 'FIBS']     |
|  2 | SJC            | San Jose       | ['Airport']             |
|  3 | DEN            | Denver         | ['Airport']             |
+----+----------------+----------------+-------------------------+


In [9]:

# Import the previously created Python DataFrame into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiport set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   df_importer.insert()

print("--")


/opt/conda/lib/python3.8/site-packages/katana/remote/aio/import_data/dataframe_importer/__init__.py:242: UserWarning: unsupported dtype 'object' for 'airport_code' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")


          0/? [?op/s]

--


In [13]:

#  What's in the graph ..

l_result1 = my_graph.query_unpaginated("""
   MATCH (n) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

#     +----+-----------------+-------------+-------------------------+------------------+------------------+--------------+
#     |    |           id(n) | labels(n)   | n.LABEL                 | n.airport_code   | n.airport_name   | n datatype   |
#     |----+-----------------+-------------+-------------------------+------------------+------------------+--------------|
#     |  0 |               1 | ['Airport'] | ['Airport']             | SJC              | San Jose         | node         |
#     |  1 |               0 | ['Airport'] | ['Airport', 'CoolTown'] | MKE              | Milwaukee        | node         |
#     |  2 |               2 | ['Airport'] | ['Airport']             | DEN              | Denver           | node         |
#     |  3 | 281474976710656 | ['Airport'] | ['Airport', 'FIBS']     | ORD              | Chicago O-Hare   | node         |
#     +----+-----------------+-------------+-------------------------+------------------+------------------+--------------+

l_result1 = my_graph.query_unpaginated("""
   MATCH (n: Airport) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

#     +----+-----------------+-------------+-------------------------+------------------+------------------+--------------+
#     |    |           id(n) | labels(n)   | n.LABEL                 | n.airport_code   | n.airport_name   | n datatype   |
#     |----+-----------------+-------------+-------------------------+------------------+------------------+--------------|
#     |  0 |               2 | ['Airport'] | ['Airport']             | DEN              | Denver           | node         |
#     |  1 |               1 | ['Airport'] | ['Airport']             | SJC              | San Jose         | node         |
#     |  2 |               0 | ['Airport'] | ['Airport', 'CoolTown'] | MKE              | Milwaukee        | node         |
#     |  3 | 281474976710656 | ['Airport'] | ['Airport', 'FIBS']     | ORD              | Chicago O-Hare   | node         |
#     +----+-----------------+-------------+-------------------------+------------------+------------------+--------------+

l_result1 = my_graph.query_unpaginated("""
   MATCH (n: CoolTown) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))



          0/? [?op/s]

+----+-----------------+-------------+-------------------------+------------------+------------------+--------------+
|    |           id(n) | labels(n)   | n.LABEL                 | n.airport_code   | n.airport_name   | n datatype   |
|----+-----------------+-------------+-------------------------+------------------+------------------+--------------|
|  0 |               2 | ['Airport'] | ['Airport']             | DEN              | Denver           | node         |
|  1 |               0 | ['Airport'] | ['Airport', 'CoolTown'] | MKE              | Milwaukee        | node         |
|  2 |               1 | ['Airport'] | ['Airport']             | SJC              | San Jose         | node         |
|  3 | 281474976710656 | ['Airport'] | ['Airport', 'FIBS']     | ORD              | Chicago O-Hare   | node         |
+----+-----------------+-------------+-------------------------+------------------+------------------+--------------+


          0/? [?op/s]

+----+-----------------+-------------+-------------------------+------------------+------------------+--------------+
|    |           id(n) | labels(n)   | n.LABEL                 | n.airport_code   | n.airport_name   | n datatype   |
|----+-----------------+-------------+-------------------------+------------------+------------------+--------------|
|  0 |               0 | ['Airport'] | ['Airport', 'CoolTown'] | MKE              | Milwaukee        | node         |
|  1 |               2 | ['Airport'] | ['Airport']             | DEN              | Denver           | node         |
|  2 |               1 | ['Airport'] | ['Airport']             | SJC              | San Jose         | node         |
|  3 | 281474976710656 | ['Airport'] | ['Airport', 'FIBS']     | ORD              | Chicago O-Hare   | node         |
+----+-----------------+-------------+-------------------------+------------------+------------------+--------------+


          0/? [?op/s]